### 畳み込み層の実装
畳み込み層の具体的な実装方法を解説します。
順伝播、逆伝播のコードを実装していく

### 畳み込み層のクラス
畳み込み層は、以下のようにクラスとして実装します。

In [ ]:
# -- 畳み込み層　 --
class ConvLayer:
    
    # n_bt : バッチサイズ、 x_ch : 入力チャンネル数、 x_h : 入力画像高さ、　x_w : 入力画像幅
    # n_flt : フィルタ数、 flt_h : フィルタ高さ、 flt_w : フィルタ幅
    # stride : スライド幅、pad : パディング幅
    # y_ch : 出力チャンネル数、　y_h : 出力高さ、　y_w : 出力幅
    
    def __init__(self, x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad):
        
        # パラメータをまとめる
        # 変更したり外部からアクセスしたりする必要のないパラメータをself.paramsにまとめられている
        self.params = (x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad)
        
        # フィルタとバイアスの初期値
        # フィルタは重みと同じように扱うことができる
        self.w = wb_width * np.random.randn(n_flt, x_ch, flt_b, flt_w)
        self.b = wb_width * np.random.randn(1, n_flt)
        
        # 出力画像のサイズ
        self.y_ch = n_flt # 出力チャンネル数
        self.y_h = (x_h - flt_h + 2*pad) // stride + 1 # 出力高さ
        self.y_w = (x_w - flt_w + 2*pad) // stride + 1 # 出力幅
        
        ...

__initi__メソッドでは、変更したり外部からのアクセスしたりする必要のないパラメータを`self.params`にまとめています。  
また、フィルタ（`self.w`）とバイアス(`b`)の初期値を設定しています。  
フィルタはニューロンの重みと同じように扱うことができるため、変数名は`w`としています。  
  
出力画像のチャンネル数、高さ、幅は、外部からアクセス可能にするために`self`が付けられています。  
このクラスに、メソッドとして順伝播と逆伝播を実装します

### 畳み込み層の順伝播
畳み込み層の順伝播を図で表すと、以下のようになります。  
  
<img src="../images/conv_forward.png">
  
この図を踏まえて、畳み込み層における順伝播を以下のようにメソッドとして実装します

In [ ]:
def forward(self, x):
    n_bt = x.shape[0]
    x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad = self.params
    y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w
    
    # 入力画像とフィルタを行列に変換
    # reshapeどういった方針でやっているのかあんま分かっていない・・
    self.cols = im2col(x, flt_h, flt_w, y_h, y_w, stride, pad)
    self.w_col = self.w.reshape(n_flt, x_ch*flt_h*flt_w)
    
    # 出力の計算 : 行列積、バイアスの加算、活性化関数
    u = np.dot(self.w_col, self.cols).T + self.b
    self.u = u.reshape(n_bt, y_h, y_w, y_ch).transpose(0, 3, 1, 2)
    # ReLU
    self.y = np.where(self.u <= 0, 0,  self.u)